In [2]:
import numpy as np

def SVD(A, threshold=1e-10):
    """
    This function compute the Singular Value Decomposition (SVD) of matrix A from scratch.
    Returns matrices U, sigma, and V_transpose such that A ≈ U @ Sigma @ V_transpose.

    Parameters:
    A : np.ndarray
        Input Matrix of shape (m, n)
    threshold : float
        Threshold below which singular values are treated as zero

    Returns:
    U : np.ndarray
        Left Singular Vectors (m x m)
    sigma : np.ndarray
        Diagonal matrix of singular values (m x n)
    V_transpose : np.ndarray
        Transpose of Right Singular vectors (n x n)
    """
    m, n = A.shape

    # Compute product of A^T and A
    B = A.T @ A

    # Find eigen values and eigen vectors of B
    eigen_values, V = np.linalg.eigh(B)

    # Sort eigen values and corresponding eigen vectors in descending order
    sorted_indices = np.argsort(eigen_values)[::-1]
    eigen_values = eigen_values[sorted_indices]
    V = V[:, sorted_indices]

    # Compute singular values
    singular_values = np.sqrt(np.clip(eigen_values, 0, None))

    # Compute U from A and V
    U = np.zeros((m, n))
    for i in range(n):
        # We are avoiding eigen vectors with extremely small or zero eigen values
        if singular_values[i] > threshold:
            U[:, i] = (A @ V[:, i]) / singular_values[i]

    # Normalize columns of U
    for i in range(n):
        norm = np.linalg.norm(U[:, i])
        if norm > threshold:
            U[:, i] /= norm

    # Compute sigma (m x n)
    sigma = np.zeros((m, n))
    np.fill_diagonal(sigma, singular_values)

    # V is already orthonormal ⇒ VT = V.T
    V_transpose = V.T

    return U, sigma, V_transpose

In [4]:
# Example matrix
A = np.array([
    [3, 1],
    [1, 3]
], dtype=float)

# Decompose A
U, sigma, V_transpose = SVD(A)

# Print the results
print("U (Left Singular Vectors):\n", U)
print("\nsigma (Diagonal Matrix of Singular Values):\n", sigma)
print("\nV_transpose (Transpose of Right Singular Vectors):\n", V_transpose)

# Calculate the original matrix using U, sigma and V_transpose to check the correctness of the decomposition
A_estimate = U @ sigma @ V_transpose
print("\nA calculated using SVD (U @ sigma @ V_transpose):\n", A_estimate)

U (Left Singular Vectors):
 [[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]

sigma (Diagonal Matrix of Singular Values):
 [[4. 0.]
 [0. 2.]]

V_transpose (Transpose of Right Singular Vectors):
 [[ 0.70710678  0.70710678]
 [-0.70710678  0.70710678]]

A calculated using SVD (U @ sigma @ V_transpose):
 [[3. 1.]
 [1. 3.]]


In [6]:
import pandas as pd
from numpy.linalg import svd

def SVD_test(num_tests: int = 10, shape: tuple = (3, 3), seed: int = 42) -> pd.DataFrame:
    """
    Generate and test SVD reconstruction accuracy on random matrices.

    Parameters:
    - num_tests: Number of random test cases
    - shape: Shape of the matrix (m, n)
    - seed: Random seed for reproducibility

    Returns:
    - DataFrame with absolute and relative reconstruction error per test
    """
    np.random.seed(seed)
    results = []
    
    for i in range(num_tests):
        A = np.random.randn(*shape)

        # Full SVD
        U, sigma, V_transpose = SVD(A)
        
        # Calculate A using SVD.
        k = min(A.shape)
        U_k = U[:, :k]                
        sigma_k = np.diag(sigma.diagonal()[:k])  
        V_k_T = V_transpose[:k, :]    
        A_estimate = U_k @ sigma_k @ V_k_T

        # Compute errors
        abs_error = np.linalg.norm(A - A_estimate, ord='fro')
        rel_error = abs_error / np.linalg.norm(A, ord='fro')

        results.append({
            "Test Case": i + 1,
            "Absolute Error": abs_error,
            "Relative Error (%)": rel_error * 100
        })

    return pd.DataFrame(results)


In [8]:
SVD_test(30, (10, 8))

,Test Case,Absolute Error,Relative Error (%)
0,1,6.395583e-15,7.452255e-14
1,2,5.748057e-15,7.047176e-14
2,3,6.479067e-15,7.024631e-14
3,4,4.496219e-15,5.239688e-14
4,5,7.425888e-15,8.922364e-14
5,6,6.024299e-15,6.293191e-14
6,7,6.930252e-15,8.495560e-14
7,8,7.958728e-15,8.475872e-14
8,9,7.223310e-15,7.832824e-14
9,10,5.173483e-15,6.030690e-14


In [10]:
SVD_test(10, (8, 10))

,Test Case,Absolute Error,Relative Error (%)
0,1,8.196012e-15,9.550148e-14
1,2,7.093132e-15,8.696251e-14
2,3,7.814974e-15,8.473027e-14
3,4,6.043777e-15,7.043141e-14
4,5,8.358877e-15,1.004337e-13
5,6,8.786484e-15,9.178665e-14
6,7,9.733987e-15,1.193256e-13
7,8,8.595189e-15,9.153690e-14
8,9,8.265370e-15,8.962816e-14
9,10,5.847756e-15,6.816685e-14


In [12]:
SVD_test(30, (8, 8))

,Test Case,Absolute Error,Relative Error (%)
0,1,5.350747e-15,7.346124e-14
1,2,1.855344e-14,2.367426e-13
2,3,6.355739e-15,8.612678e-14
3,4,6.095640e-15,7.086067e-14
4,5,1.256761e-14,1.680040e-13
5,6,1.093405e-14,1.550560e-13
6,7,7.635678e-15,8.850847e-14
7,8,5.869146e-15,7.183509e-14
8,9,4.453132e-15,5.713535e-14
9,10,1.142748e-14,1.428800e-13
